# Check Which Stocks Have Most News Available

In [1]:
import pandas as pd
from tqdm.notebook import tqdm

# Load dataset
DATA_SIZE = 15549298
CHUNK_SIZE = 10000
nasdaq_path = "news_data/direct_data/nasdaq_external_data.csv"
nasdaq_df = pd.concat(
    tqdm([
        chunk.dropna() for chunk in tqdm(
            pd.read_csv(
                nasdaq_path,
                index_col="Date",
                usecols=["Date", "Stock_symbol"],
                dtype=str,
                chunksize=CHUNK_SIZE),
            total=DATA_SIZE//CHUNK_SIZE + 1,
            desc="Chunks Read")],
        total=DATA_SIZE//CHUNK_SIZE + 1,
        desc="Chunks Put Together"
        ))

nasdaq_df.dropna(inplace=True)
stock_list = nasdaq_df["Stock_symbol"].value_counts()
stock_list.head(50)

Chunks Read:   0%|          | 0/1555 [00:00<?, ?it/s]

Chunks Put Together:   0%|          | 0/1555 [00:00<?, ?it/s]

Stock_symbol
GILD     12376
NVDA     11862
QQQ      11813
BABA     11625
WFC      11301
INTC     11157
MRK      10774
TSLA     10587
KO       10521
BROGW    10456
BPYPO     9979
GOOG      9930
DIS       9654
BHFAL     9614
MU        9605
T         9463
MS        9458
AAPL      9338
AMD       9209
PMAY      9108
CLSN      9106
ACGLO     9014
DNOV      8990
GSEE      8966
QCOM      8954
FDEV      8891
OCFCP     8832
PFFL      8822
CMCSA     8820
BRK       8797
FDX       8744
MSFT      8737
GS        8730
HCRB      8722
FFEB      8718
UEVM      8715
BSMR      8708
UCIB      8699
WLDR      8698
KJUL      8693
CVX       8688
WMT       8686
GDMA      8680
WTRE      8680
GE        8680
ARTLW     8677
FSMB      8662
CGRO      8640
HVT-A     8630
UFEB      8603
Name: count, dtype: int64

# Get Set of All Stocks

In [2]:
# Getting list of all stocks for which we have sufficient data
stock_list = stock_list[stock_list > 9000]
stocks = list(stock_list.index)

# Get Date Range

In [3]:
# Get min and max datetimes for each ticker
nasdaq_df.index = pd.to_datetime(nasdaq_df["date"], errors="coerce")
ticker_date_summary = nasdaq_df.groupby("Stock_symbol")["date"].agg(["min", "max"]).reset_index()


# Filter out unnecessary
ticker_date_summary = ticker_date_summary[ticker_date_summary["Stock_symbol"].isin(stocks)]

# Only keep date in datetime
ticker_date_summary["start"] = pd.to_datetime(ticker_date_summary["min"]).dt.date
ticker_date_summary["end"] = pd.to_datetime(ticker_date_summary["max"]).dt.date
ticker_date_summary.drop(columns=["min", "max"], inplace=True)

# Fix indexing
ticker_date_summary.set_index("Stock_symbol", inplace=True)
ticker_date_summary = ticker_date_summary.rename_axis("ticker")

ticker_date_summary

KeyError: 'date'

In [ ]:
import os

# Make directory
out_dir = "stock_data"
os.mkdir(out_dir, exist_ok=True)

ticker_date_summary.to_csv(os.path.join(out_dir, "stock_list.csv"))

print("✅ Viable stocks table has been exported")